In the Data Preprocessing section, you will learn data preprocessing techniques with scikit-learn, one of the most popular frameworks used for industry data science.

The scikit-learn library includes tools for data preprocessing and data mining. It is imported in Python via the statement `import sklearn`.